# Scraper


In [14]:
#################################
# DEPENDENCIES:
#################################
from splinter import Browser
from bs4 import BeautifulSoup
import time
import pandas as pd

In [8]:
#################################
# FUNCTION TO INITIALIZE BROWSER USING CHROME DRIVER 
# - LOCATED IN THE SAME FOLDER
#################################
def init_browser():
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

## RF!! headless=True when ready to submit!

In [9]:
#################################
# FUNCTION TO SCRAPE NASA
# NASA
# https://mars.nasa.gov/news/
#################################
# def scrape_nasa():
browser = init_browser()

nasa_dict = {
    "news_title": "",
    "news_p": ""
}


url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

nasa_dict["news_title"] = soup.find("div", class_="content_title").get_text()
nasa_dict["news_p"] = soup.find("div", class_="article_teaser_body").get_text()


browser.quit()

#     return nasa_dict


# VALIDATE:
print(nasa_dict["news_title"])
print(nasa_dict["news_p"])

Space Samples Link NASA's Apollo 11 and Mars 2020
While separated by half a century, NASA's Apollo 11 and Mars 2020 missions share the same historic goal: returning samples to Earth.


In [11]:
#################################
# FUNCTION TO SCRAPE JPL
# JPL
# https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars
#################################
# def scrape_jpl():
browser = init_browser()

url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, "html.parser")


raw_ending_url = soup.find("div", class_="carousel_items").find("article")["style"] 
# check the result:
# raw_ending_url


ending_url_list = raw_ending_url.split("'")
# check the result:
# ending_url_list[1]


base_url = "https://www.jpl.nasa.gov"
featured_image_url = base_url + ending_url_list[1]


browser.quit()

# VALIDATE:
print(featured_image_url)


# return featured_image_url

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA09320-1920x1200.jpg


In [12]:
##################################################################
# SKIP: CANNOT BE DONE DUE TO TWITTER ENCRYPTION 
##################################################################
# FUNCTION TO SCRAPE MARS WEATHER
# MWEATHER
# https://twitter.com/marswxreport?lang=en

# Visit the Mars Weather twitter account here and scrape the latest Mars 
# weather tweet from the page. Save the tweet text for the weather report 
# as a variable called mars_weather.
#################################


In [28]:
#################################
# FUNCTION TO SCRAPE MARS FACTS
# MFACTS
# https://space-facts.com/mars/

# Mars Facts

# Visit the Mars Facts webpage here and use Pandas to scrape 
# the table containing facts about the planet including Diameter, Mass, etc.

# Use Pandas to convert the data to a HTML table string.
#################################
url = "https://space-facts.com/mars/"

# Use PANDAS to scrape:
tableslist = pd.read_html(url)

table_df = tableslist[1]


# if necessary: named the columns:
final_table_df = table_df.rename(columns={
    0: 'fact',
    1: 'value'
})

# VALIDATE:
final_table_df



# Use PANDAS to convert data to HTML table string: 
html_table = final_table_df.to_html(index=False)
html_table.replace('\n', '')


'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>fact</th>      <th>value</th>    </tr>  </thead>  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [79]:
#################################
# FUNCTION TO SCRAPE MARS HEMISPHERE
# MHEMI
# https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
#################################
browser = init_browser()

# This list will contain one dictionary for each hemisphere:
hemisphere_image_urls = []


url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, "html.parser")


linkresults = soup.find_all('a', class_='itemLink')

#################################
# Pull out the image partial links and their titles:
#################################
for x in linkresults:
    
    raw_title = x.find('img')
    link = x['href']
    
    if raw_title and link:
        title = raw_title['alt']
        final_title = title.replace(" Enhanced thumbnail", "")
        
        # click on the link to retrieve the the full res image url:
        browser.click_link_by_partial_text(final_title)
        time.sleep(1)
        html2 = browser.html
        soup2 = BeautifulSoup(html2, "html.parser")
        
        
        newresults = soup2.find_all('a')
        # loop through the a tags to find the right link:
        for hemi in newresults:
            if hemi.text == 'Sample':
                newlink = hemi['href']
                

        #Use a Python dictionary to store the data using the keys img_url and title:
        temp_dict = {
            "title": final_title,
            "img_url": newlink
        }

        # Append the dictionary with the image url string and the hemisphere title to a list
        hemisphere_image_urls.append(temp_dict)


#################################
# Clean up:
#################################
browser.quit()


# VALIDATE:
hemisphere_image_urls
      

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_unenhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_unenhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_unenhanced.tif/full.jpg'}]

In [77]:
#################################
# TO TEST THE NESTED FOR LOOP ABOVE
#################################
browser = init_browser()
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)


browser.click_link_by_partial_text('Hemisphere')
# browser.click_link_by_partial_href('/search/map/Mars/Viking/cerberus_enhanced')

time.sleep(1)
html2 = browser.html
soup2 = BeautifulSoup(html2, "html.parser")

newresults = soup2.find_all('a')

for x in newresults:
    if x.text == "Sample":
        print(x['href'])



browser.quit()

newresults

http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg


[<a href="https://www.usgs.gov/centers/astrogeo-sc" style="float:right;margin-top:10px;">
 <img alt="USGS: Science for a Changing World" class="logo" height="60" src="/images/usgs_logo_main_2x.png"/>
 </a>,
 <a href="https://nasa.gov" style="float:right;margin-top:5px;margin-right:20px;">
 <img alt="NASA" class="logo" height="65" src="/images/logos/nasa-logo-web-med.png"/>
 </a>,
 <a href="https://pds-imaging.jpl.nasa.gov/" style="float:right;margin-top:5px;margin-right: 10px;">
 <img alt="PDS Cartography and Imaging Science Node" class="logo" height="65" src="/images/pds_logo-invisible-web.png"/>
 </a>,
 <a href="/search" style="float:right;text-decoration:none;">
 <img alt="Astropedia" src="/images/astropedia/astropedia-logo-main.png" style="width:200px;border:none;float:right;"/>
 <div style="clear:both;font-size:.8em;float:right;color:#888;">Lunar and Planetary Cartographic Catalog</div>
 </a>,
 <a href="http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced